In [ ]:
here it takes about 8.5 mins for scrolling down and then overall 16.37 mins
# last height: 439621
# new height: 439620
#    ---> 496 total scrolls
# df.to_csv('reviews.csv',index=False)

In [8]:
#this is the oops concept code this takes 18 minutes to run and scrape all data
#have not corrected the formats for dates and this isnt uniform as i have used different selectors to show various ways to do it and efficiency could be improved
#all metrics work perfect as requirement and oops concepts have been implimented
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

class ProductScraper:
    def __init__(self, url):
        self.url = url
        self.chrome_options = Options()
        self.chrome_options.add_argument('--headless')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(executable_path='chromedriver.exe', options=self.chrome_options)
        self.name_list = []
        self.verified_list = []
        self.stars_list = []
        self.title_list = []
        self.review_list = []
        self.duration_list = []
        self.extra_list = []

    def navigate_to_page(self):
        self.driver.get(self.url)
        time.sleep(2)

    def extract_product_information(self):
        ele = self.driver.find_element_by_xpath('//*[@id="__nuxt"]')
        brand = ele.find_element_by_xpath('//*[@id="product-info"]')
        price = ele.find_element_by_xpath('//*[@id="main"]/div/div[1]/div[1]/div/p')
        description = ele.find_element_by_xpath('//*[@id="main"]/div/div[1]/div[3]/div[4]')
        count = ele.find_element_by_class_name('okeReviews-reviewsSummary-ratingCount')
        print(brand.text)
        print(price.text)
        print(count.text)
        print(description.text)

    def scroll_to_load_reviews(self):
        SCROLL_PAUSE_TIME = 1
        last_height = self.driver.execute_script("return document.body.scrollHeight")

        ele = self.driver.find_element_by_xpath('//*[@id="__nuxt"]')
        number = 0
        print(f'last height: {last_height}')
        
        while True:
            number = number + 1
            self.driver.execute_script('arguments[0].scrollBy(0, 3000);', ele)
            time.sleep(SCROLL_PAUSE_TIME)

            new_height = self.driver.execute_script("return arguments[0].scrollHeight", ele)

            print(f'new height: {new_height}')
            print(f"   ---> {number} total scrolls")

            if new_height < last_height:
#             if number < 2:
                try:
                    item = self.driver.find_element_by_xpath('//*[@id="reviews"]')
#                     time.sleep(SCROLL_PAUSE_TIME)
                    button = item.find_element_by_class_name('okeReviews-showMore')
                    if button:
                        button.click()
                        print("clicked")
                except NoSuchElementException:
                    print("No 'Show More' button found. Exiting the loop.")
                    break

                print('cont')
            last_height = new_height + 1
            self.driver.execute_script('arguments[0].scrollBy(0, 3000);', ele)

    def extract_reviews(self):
        primary_elements = self.driver.find_elements_by_class_name("okeReviews-review")
        for review in primary_elements:
            name_element = review.find_element_by_class_name("okeReviews-review-reviewer-profile-name")
            name = name_element.text
            try:
                verified_element = review.find_element_by_class_name("okeReviews-review-reviewer-profile-status--verified")
                if verified_element:
                    verified = verified_element.text
            except NoSuchElementException:
                verified = "Not Verified"
                print("No verification found.")


            stars_element = review.find_element_by_class_name("okeReviews-review-starRating")
            stars = stars_element.text

            title_element = review.find_element_by_class_name("okeReviews-review-main-heading")
            title = title_element.text

            review_elements = review.find_elements_by_class_name("okeReviews-review-main-content")
            if review_elements:
                reviews = review_elements[0].text
            else:
                reviews = "NA"

            duration_element = review.find_element_by_class_name("okeReviews-review-date")
            duration = duration_element.text
            duration_parts = duration.split('\n')
            if len(duration_parts) > 1:
                duration = duration_parts[1]
            else:
                duration = "NA"

            extra_element = review.find_elements_by_class_name("okeReviews-review-reviewer-attributes")
            if extra_element:
                extra = extra_element[0].text
                string = extra.split('\n')
                extra = ','.join(string)
            else:
                extra = "NA"

            self.name_list.append(name)
            self.verified_list.append(verified)
            self.stars_list.append(stars)
            self.title_list.append(title)
            self.review_list.append(reviews)
            self.duration_list.append(duration)
            self.extra_list.append(extra)

    def create_dataframe(self):
        self.df = pd.DataFrame({
            'name': self.name_list,
            'verification': self.verified_list,
            'rating': self.stars_list,
            'title': self.title_list,
            'review': self.review_list,
            'date of review': self.duration_list,
            'details': self.extra_list
        })

    def export_to_csv(self, filename='reviews.csv'):
        self.df.to_csv(filename, index=False)
        print(f"CSV File '{filename}' Created")

    def close_driver(self):
        self.driver.quit()


# Example Usage:
url = 'website here'
scraper = ProductScraper(url)
scraper.navigate_to_page()
scraper.extract_product_information()
scraper.scroll_to_load_reviews()
scraper.extract_reviews()
scraper.create_dataframe()
scraper.export_to_csv()
scraper.close_driver()

Balmy Tint Hydrating Lip Balm
$28 USD
1242 Reviews
Description
What it is: A clean, tinted lip balm with active levels of moisture-boosting ingredients and sheer, buildable color.
How it works: The buttery, balmy formula quenches lips on contact while adding a kiss of color that builds with each swipe. Salicornia (Sea Succulent), Rosehip Seed Oil, and Shea Butter deeply hydrate and nourish, helping lips look smoother and more plump.
What else you need to know: The cult-classic lip balm that started it all, reimagined. Each Balmy Tint is carefully poured by hand to maintain the integrity of the active formula. Available in eight easy-to-wear shades for an effortless wash of color that you can dial up or down.
Size: 0.15 oz | 4.4 g
Awards:
- Best Lip Balm, Glamour Beauty Awards (2022)
- Best Tinted Lip Balm, Refinery29's Beauty Innovator Awards (2021)
- POPSUGAR Pick (2021)
last height: 4591
new height: 6727
   ---> 1 total scrolls
new height: 6727
   ---> 2 total scrolls
clicked
cont
ne

new height: 115642
   ---> 144 total scrolls
new height: 115642
   ---> 145 total scrolls
clicked
cont
new height: 117539
   ---> 146 total scrolls
new height: 117539
   ---> 147 total scrolls
clicked
cont
new height: 117539
   ---> 148 total scrolls
clicked
cont
new height: 117539
   ---> 149 total scrolls
clicked
cont
new height: 121273
   ---> 150 total scrolls
new height: 121273
   ---> 151 total scrolls
clicked
cont
new height: 121273
   ---> 152 total scrolls
clicked
cont
new height: 123169
   ---> 153 total scrolls
new height: 124889
   ---> 154 total scrolls
new height: 124889
   ---> 155 total scrolls
clicked
cont
new height: 124889
   ---> 156 total scrolls
clicked
cont
new height: 126785
   ---> 157 total scrolls
new height: 128564
   ---> 158 total scrolls
new height: 128564
   ---> 159 total scrolls
clicked
cont
new height: 128564
   ---> 160 total scrolls
clicked
cont
new height: 132209
   ---> 161 total scrolls
new height: 132209
   ---> 162 total scrolls
clicked
cont
ne

clicked
cont
new height: 273037
   ---> 302 total scrolls
new height: 273037
   ---> 303 total scrolls
clicked
cont
new height: 274874
   ---> 304 total scrolls
new height: 274874
   ---> 305 total scrolls
clicked
cont
new height: 276623
   ---> 306 total scrolls
new height: 276623
   ---> 307 total scrolls
clicked
cont
new height: 278431
   ---> 308 total scrolls
new height: 278431
   ---> 309 total scrolls
clicked
cont
new height: 280357
   ---> 310 total scrolls
new height: 280357
   ---> 311 total scrolls
clicked
cont
new height: 282420
   ---> 312 total scrolls
new height: 282420
   ---> 313 total scrolls
clicked
cont
new height: 284337
   ---> 314 total scrolls
new height: 284337
   ---> 315 total scrolls
clicked
cont
new height: 286522
   ---> 316 total scrolls
new height: 286522
   ---> 317 total scrolls
clicked
cont
new height: 288389
   ---> 318 total scrolls
new height: 288389
   ---> 319 total scrolls
clicked
cont
new height: 290315
   ---> 320 total scrolls
new height: 290

new height: 401239
   ---> 461 total scrolls
clicked
cont
new height: 402637
   ---> 462 total scrolls
new height: 402637
   ---> 463 total scrolls
clicked
cont
new height: 404203
   ---> 464 total scrolls
new height: 404203
   ---> 465 total scrolls
clicked
cont
new height: 406397
   ---> 466 total scrolls
new height: 406397
   ---> 467 total scrolls
clicked
cont
new height: 408436
   ---> 468 total scrolls
new height: 408436
   ---> 469 total scrolls
clicked
cont
new height: 409884
   ---> 470 total scrolls
new height: 409884
   ---> 471 total scrolls
clicked
cont
new height: 411594
   ---> 472 total scrolls
new height: 411594
   ---> 473 total scrolls
clicked
cont
new height: 413434
   ---> 474 total scrolls
new height: 413434
   ---> 475 total scrolls
clicked
cont
new height: 414856
   ---> 476 total scrolls
new height: 414856
   ---> 477 total scrolls
clicked
cont
new height: 416250
   ---> 478 total scrolls
new height: 416250
   ---> 479 total scrolls
clicked
cont
new height: 417